# Categorical Variables

Open in Colab: [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/febse/econ2025/blob/main/09-Categorical-Variables.ipynb)

## Simpson's paradox

The data set `gdp2019` contains the GDP per capita in current prices for 142 countries in 2019. In addition, the variable `spending` shows the government spending in these countries as a share (in percent) of GDP.

-   `gdppc` (numeric): [GDP per capita](https://www.imf.org/external/datamapper/NGDPDPC@WEO/OEMDC/ADVEC/WEOWORLD) in USD (current prices)
-   `spending` (numeric): [Government spending as a share of GDP](https://www.imf.org/external/datamapper/exp@FPP/USA/FRA/JPN/GBR/SWE/ESP/ITA/ZAF/IND).

The data set was constructed using data from the IFM. You can find more information about the two variables by following the links above.

We want to explore the relationship between `gdppc` and `spending`.


In [ ]:
library(tidyverse)

## Simpson's paradox (1)

berkley <- read_csv(
  "https://waf.cs.illinois.edu/discovery/berkeley.csv"
  ) %>%
  mutate(
    is_rejected = ifelse(Admission == "Accepted", 0, 1),
  )

berkley %>% head()

In [ ]:
berkley %>%
  group_by(Gender) %>%
    summarize(
        n = n(),
        n_rejected = sum(is_rejected),
        p_rejected = round(mean(is_rejected), 2)
    )

In [ ]:
admission_by_major_gender <- berkley %>%
  group_by(Major, Gender) %>%
    summarize(
        n = n(),
        n_rejected = sum(is_rejected),
        p_rejected = round(mean(is_rejected), 2)
    )
admission_by_major_gender

In [ ]:
admission_by_major_gender %>%
  ggplot(aes(x = Major, y = p_rejected, fill=Gender)) +
    geom_bar(stat = "identity", position = "dodge") +
    labs(
      y = "Admission rate",
      x = "Major"
    ) +
    theme(axis.text.x = element_text(angle = 45, hjust = 1))

In [ ]:
berkley %>% head()

In [ ]:
model_matrix <- model.matrix(lm(is_rejected ~ 0 + Gender, data = berkley))
model_matrix %>% head()

In [ ]:
berkey <- berkley %>%
  bind_cols(model_matrix)
berkey %>% head()

In [ ]:
lm(is_rejected ~ Gender, data = berkley) %>%
  summary()

## GDP per Capita and Government Spending

The data set `gdp2019` contains the GDP per capita in current prices for 142 countries in 2019. In addition, the variable `spending` shows the government spending in these countries as a share (in percent) of GDP.

-   `gdppc` (numeric): [GDP per capita](https://www.imf.org/external/datamapper/NGDPDPC@WEO/OEMDC/ADVEC/WEOWORLD) in USD (current prices)
-   `spending` (numeric): [Government spending as a share of GDP](https://www.imf.org/external/datamapper/exp@FPP/USA/FRA/JPN/GBR/SWE/ESP/ITA/ZAF/IND).

The data set was constructed using data from the IFM. You can find more information about the two variables by following the links above.

We want to explore the relationship between `gdppc` and `spending`.

In [ ]:
gdp2019 <- read_csv(
  "https://raw.githubusercontent.com/feb-sofia/econometrics-2023/main/data/gdpgov2019.csv"
  ) %>%
  filter(!is.na(spending))
  
gdp2019 %>% head()

1.  Create a scatterplot for the two variables and add the estimated regression line for the model

$$
\text{gdppc}_i = \beta_0 + \beta_1 \text{spending}_i + e_i, \quad e_i \sim N(0, \sigma^2)
$$


In [ ]:
gdp2019 %>%
  ggplot(aes(x = spending, y = gdppc)) +
  geom_point() +
  geom_smooth(method = "lm") +
  labs(
    y = "GDP per capita",
    x = "Gov. spending"
  ) +
  lims(
    x = c(0, 70),
    y = c(0, 90000)
  )

In [ ]:
# Estimate the model


2.  Create a new variable called `gdppc_gr` that has five categories:

-   Low: if $\text{gdppc} \leq 1025$
-   Medium-low: if $1025 < \text{gdppc} \leq 3995$
-   Medium-high: if $3995 < \text{gdppc} \leq 12375$
-   High: if $12375 < \text{gdppc} \leq 30000$
-   Very high: if $\text{gdppc} > 30000$

In [ ]:
gdp2019 <- gdp2019 %>%
  mutate(
    gdppc_gr = cut(
      gdppc,
      breaks = c(0, 1025, 3995, 12375, 30000, Inf),
      labels = c("Low", "Medium-low", "Medium-high", "High", "Very high"))
  )
table(gdp2019$gdppc_gr)

4.  Estimate the following linear model and interpret the estimated coefficients

$$
\text{gdppc}_i = \beta_0 + \beta_{\text{gdppc\_gr}[i]} + e_i, e_i \sim N(0, \sigma^2)
$$

In [ ]:
gdp2019 %>%
  ggplot(aes(x = spending, y = gdppc)) +
  geom_point(aes( color = gdppc_gr)) +
  geom_smooth(method = "lm") +
  # geom_abline(
  #   intercept = c(-144.39, 1308.32, 5835.94, 18460.21, 51920.69),
  #   slope = 44.80,
  #   alpha = 0.5
  # ) +
  lims(
    x = c(0, 80),
    y = c(0, 90000)
  )

## F-Test

Very often we want test the hypothesis that a certain subset of the coefficients are simultaneously equal to zero. This is called a joint hypothesis.

In our example we could test the hypothesis that the coefficients of `Med Low`, `Med High`, `High` and `Very High` are all equal to zero. This is equivalent to testing if the average of `gdppc` is the same in all groups.

The test is called an F-test and is based on the ratio of the sum of squared errors of the restricted model and the sum of squared errors of the unrestricted model.

The restricted model is the one where the coefficients of `Med Low`, `Med High`, `High` and `Very High` are all equal to zero.

The unrestricted model is the one where all coefficients are estimated.

The F-test is based on the following statistic:

$$
F = \frac{(RSS_{\text{restricted}} - RSS_{\text{unrestricted}}) / (p_{\text{restricted}} - p_{\text{unrestricted}})}{RSS_{\text{unrestricted}} / (n - p_{\text{unrestricted}})}
$$

$$
F = \frac{(RSS_{\text{restricted}} - RSS_{\text{unrestricted}})}{RSS_{\text{unrestricted}}}\frac{(n - p_{\text{unrestricted}})}{(p_{\text{restricted}} - p_{\text{unrestricted}})}
$$

where $RSS$ is the residual sum of squares, $p$ is the number of parameters in the model and $n$ is the number of observations.